# City street network orientations

Compare the spatial orientations of city street networks.

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  - [Journal article/citation](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [15]:
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd

ox.config(log_console=True, use_cache=True)
weight_by_length = False

In [16]:
# define the study sites as label : query
places = {'Viti Levu'       : 'Viti Levu, Fiji',
          'Isatabu'        : 'Guadalcanal, Solomon Islands',
          'Espiritu Santo'       : 'Espiritu Santo, Vanuatu',
          'Grande Terre'     : 'Grande Terre, New Caledonia',
          'Savai\'i'       : 'Savai\'i, Samoa',
          'Tahiti':'Tahiti, French Polynesia',
          'Guam':'Guam, United States of America',
          'Tarawa':'Tarawa, Kiribati',
          'Tongatapu':'Tongatapu, Tonga',
          'Pohnpei':'Pohnpei, Federated States of Micronesia',
          'Koror':'Koror, Palau',
          'Funafuti':'Funafuti, Tuvalu'
          }

In [17]:
# verify OSMnx geocodes each query to what you expect
gdf = ox.gdf_from_places(places.values())
gdf

,bbox_east,bbox_north,bbox_south,bbox_west,geometry,place_name
0,178.695794,-17.311033,-18.275698,177.243045,"POLYGON ((177.2430451 -17.9628669, 177.2431481...","Viti Levu, Western, Viti"
1,160.834167,-9.250366,-9.946397,159.583020,"(POLYGON ((159.5830204 -9.3674043, 159.5833766...","Guadalcanal Island, Aola, Guadalcanal Province..."
2,167.259715,-14.643440,-15.668947,166.541482,"POLYGON ((166.5414822 -14.8457294, 166.5415036...","Santo, Sanma, Vanuatu"
3,167.033359,-20.075598,-22.395544,163.995351,"POLYGON ((163.9953515 -20.1004952, 163.9954806...","Grande-Terre, Poya, Province Nord, Nouvelle-Ca..."
4,-172.174036,-13.438806,-13.803698,-172.804067,"POLYGON ((-172.8040668 -13.515888, -172.803986...","Savai'i, Apia, Sāmoa"
5,-149.124707,-17.494194,-17.880416,-149.620873,"POLYGON ((-149.6208735 -17.5623658, -149.62069...","Tahiti, Teva I Uta, Îles du Vent, Polynésie Fr..."
6,145.009167,13.706179,13.182335,144.563426,"POLYGON ((144.563426 13.448065, 144.56355 13.4...","Guam, United States of America"
7,173.166919,1.641935,1.323707,172.898623,"POLYGON ((172.8986226 1.5224277, 172.9036049 1...","Tarawa, North Tarawa, Kiribati"
8,-174.761087,-20.813108,-22.549080,-176.433766,"(POLYGON ((-176.4337657 -22.3352885, -176.4303...","Tongatapu, Tonga"
9,160.917151,8.348000,0.827000,154.039955,"(POLYGON ((154.0399549 8.101000000000001, 154....","Pohnpei, Micronesia"


## Get the street networks and their edge bearings

In [18]:
bearings = {}
for place in sorted(places.keys()):
    
    # get the graph
    query = places[place]
    G = ox.graph_from_place(query, network_type='drive')
    
    # calculate edge bearings
    G = ox.add_edge_bearings(G)
    
    if weight_by_length:
        # weight bearings by length (meters)
        streets = [(d['bearing'], int(d['length'])) for u, v, k, d in G.edges(keys=True, data=True)]
        city_bearings = []
        for street in streets:
            city_bearings.extend([street[0]] * street[1])
        bearings[place] = pd.Series(city_bearings)
    else:
        # don't weight bearings, just take one value per street segment
        bearings[place] = pd.Series([data['bearing'] for u, v, k, data in G.edges(keys=True, data=True)])

## Visualize it

In [19]:
# function to draw a polar histogram for a set of edge bearings
def polar_plot(ax, bearings, n=36, title=''):

    bins = [ang * 360 / n for ang in range(0, n + 1)]
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width =  2 * np.pi / n

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')

    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
                  color='#003366', edgecolor='k', linewidth=0.5, alpha=0.7)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family':'Calibri', 'size':26, 'weight':'bold'}
    xtick_font = {'family':'Calibri', 'size':12, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family':'Calibri', 'size':10, 'weight':'bold', 'alpha':0.3, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [20]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # this prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = [ang * 360 / n for ang in range(0, n + 1)]
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = count.tolist()
    count = [count[-1]] + count[:-1]

    count_merged = []
    count_iter = iter(count)
    for count in count_iter:
        merged_count = count + next(count_iter)
        count_merged.append(merged_count)

    return np.array(count_merged)

In [23]:
# create figure and axes
n = len(places)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})
axes = [item for sublist in axes for item in sublist]

# plot each city's polar histogram
for ax, place in zip(axes, sorted(places.keys())):
    polar_plot(ax, bearings[place], title=place)

# add super title and save full image
suptitle_font = {'family':'Open Sans', 'fontsize':60, 'fontweight':'light', 'y':1.07}
fig.suptitle('Pacific Island Street Network Orientations', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
plt.gcf().savefig('images/street-orientations.png', dpi=120, bbox_inches='tight')
plt.close()